In [1]:
import numpy as np
import scipy as sp
from scipy import io,integrate,sparse
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.lines import Line2D
import matplotlib.patches as mpatches
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from mpl_toolkits.axes_grid1.inset_locator import mark_inset

from kpm_bin import *

from IPython.display import clear_output
np.set_printoptions(linewidth=300)
%load_ext autoreload

%autoreload 2

## TeX preamble
preamble = r"""
\usepackage[no-math]{fontspec}
\setmainfont[]{Helvetica Condensed}

\usepackage{amsmath,amssymb,amsthm}
\usepackage{mathastext}
"""

mpl.use("pgf")

mpl.rcParams.update({
    "pgf.texsystem": "lualatex",
    'font.family': 'serif',
    'font.size' : 8,
    'text.usetex': True,
    'pgf.rcfonts': False,
    'pgf.preamble': preamble,
})

In [2]:
H = sp.io.mmread('matrices/Ga41As41H72.mtx')
H.tocsr()

d = H.shape[0]

In [3]:
%%time
Emin_true = sp.sparse.linalg.eigsh(H,k=1,which='SA',tol=1e-3,ncv=100,return_eigenvectors=False)[0]
Emax_true = sp.sparse.linalg.eigsh(H,k=1,which='LA',tol=1e-3,ncv=100,return_eigenvectors=False)[0]

CPU times: user 1min 29s, sys: 712 ms, total: 1min 30s
Wall time: 22.8 s


In [4]:
Emin_true,Emax_true

(-1.2502010779156862, 1300.9339157082688)

In [5]:
m = 10
k = 400
np.random.seed(0)

Rvs = []
Rws = []
αβs = []
for _ in range(m):
    print(f'random sample {_}')
    clear_output(wait=True)

    v = np.random.randn(d)
    v /= np.linalg.norm(v)

    (α,β) = lanczos(H,v,k)
    αβs.append((α,β))

    Hk = np.diag(β,1) + np.diag(β,-1) + np.diag(np.append(α,0))

    Rv,Rvec = np.linalg.eigh(Hk[:k,:k])
    Rvs.append(Rv)
    Rws.append(Rvec[0]**2)
    
Rvs = np.array(Rvs)
Rws = np.array(Rws)

random sample 9


In [6]:
# bins = np.hstack([[-.00001,.00001,800],np.linspace(intervals[-1][0],intervals[-1][1],65)])
# plt.xlim(1298,1302)
# plt.ylim(0,1e-3)
# plt.hist(Rvs.flatten(),weights=Rws.flatten(),density=True,bins=bins);
# plt.hist(np.hstack([np.zeros(d-123),Evs_top]),bins=bins,density=True,alpha=.5);


In [7]:
Emin_true = np.min(Rvs,axis=0)[0]
Emax_true = np.max(Rvs,axis=-1)[0]

Er = np.max(Rvs[Rvs<1000])
El = np.min(Rvs[Rvs>1000])

In [8]:
Evs_top = sp.sparse.linalg.eigsh(H,k=123,which='LA',tol=1e-4,ncv=330,return_eigenvectors=False)

In [9]:
intervals_raw = np.array([[Emin_true,Er],[El,Emax_true]])

In [10]:
methods = ['T','Thd','Uspike']

k_eff = min(200,2*k)

ϵs = [0]

ϵ = 1e-4
Emin = Emin_true - ϵ*(Emax_true - Emin_true)
Emax = Emax_true + ϵ*(Emax_true - Emin_true)

EE = np.linspace(-10,90,1000)
EE = np.hstack([EE,np.linspace(1298,1302,500)])
EE = np.sort(EE)

fig,axs = plt.subplots(2,1,figsize=(6,6))
plt.subplots_adjust(hspace=.3,bottom=.12,top=.95)
# fig,axs = plt.subplots(1,2,figsize=(12,4))
# plt.subplots_adjust(wspace=.3)

# axs[1] = inset_axes(axs, "100%","100%", loc='upper left', bbox_to_anchor=(0.45,.35,.5,.6), bbox_transform=axs.transAxes) # zoom = 6
# axs[1].plot()


for i,method in enumerate(methods):
    
    for ϵ in ϵs:
        Emin = Emin_true - ϵ*(Emax_true - Emin_true)
        Emax = Emax_true + ϵ*(Emax_true - Emin_true)
    
        μs = []
    
        if method == 'Uspike':
            ϵl = ϵ
            ϵr = .1
            intervals = np.array([[Emin_true-ϵl*(Er-Emin_true),Er+ϵl*(Er-Emin_true)],\
                                  [El-ϵr*(Emax_true-El),Emax_true+ϵr*(Emax_true-El)]])

            αβ0s = [get_chebT_recurrence(2*k,int[0],int[1]) for int in intervals]

            wl = .95
            wr = 1-wl
            weights = [wl,wr]
            
            (γ,δ) = get_multiint_op_recurrence(αβ0s,weights,2*k)
            σ = lambda x: wl*get_chebT_density(*intervals[0])(x) + wr*get_chebT_density(*intervals[1])(x)

          
        elif method =='T' or method == 'Thd':
            (γ,δ) = get_chebT_recurrence(2*k,Emin,Emax)
            σ = get_chebT_density(Emin,Emax)
        
        for (α,β) in αβs:
    
            Hk = np.diag(β,1) + np.diag(β,-1) + np.diag(np.append(α,0))
            e0 = np.zeros(k+1)
            e0[0] = 1
        
            μ = get_moments(Hk,e0,2*k,γ,δ)
    
            μs.append(μ)


        k_eff_loc = k_eff
        if method == 'Thd':
            k_eff_loc = 800
        g = np.ones(k_eff_loc)
        # if method == 'T' or method == 'Thd':
        #     g = jackson_weights(k_eff_loc)
        
        dρdσ = get_op_expansion(g*np.mean(μs,axis=0)[:k_eff_loc],γ,δ)
        ρ_KPM = lambda E: σ(E)*dρdσ(E)

        styles = [{'color':plt.cm.magma(.85),'lw':1,'alpha':1},\
                  {'color':plt.cm.magma(.5),'lw':1,'alpha':1},\
                  {'color':plt.cm.magma(.12),'lw':1,'alpha':1}]

        axs[0].plot(EE,ρ_KPM(EE),**styles[i])
        kernel = np.exp(-np.linspace(-1,1,10)**2)
        kernel /= np.sum(kernel)
        axs[1].plot(EE,np.convolve(ρ_KPM(EE),kernel,mode='same'),**styles[i])

bins = np.linspace(1299,1301,40)
weights = np.ones(123)/(d*(bins[-1]-bins[-2]))
axs[1].hist(Evs_top,bins=bins,weights=weights,histtype='stepfilled',lw=.5,ec='black',fc='#dddddd')
#axs[1].hist(np.hstack([1297*np.ones(d-123),Evs_top]),bins=bins,density=True,histtype='stepfilled',lw=.5,ec='black',fc='#dddddd')
#axs[1].hist(np.hstack([np.zeros(d-123),Evs_top]),bins=bins,density=True,histtype='bar',fc='#dddddd')

axs[0].set_ylim(-.001,.04)
axs[0].set_xlim(-10,80)

# # sub region of the original image
x1, x2, y1, y2 = 1299,1301,-2e-4,1.5e-3
axs[1].set_xlim(x1, x2)
axs[1].set_ylim(y1, y2)
# axs[1].set_xticks([x1,x2])
# axs[1].set_yticks([y1,y2])


for ax in axs:
    ax.set_xlabel('energy \emph{E}')
    ax.set_ylabel('density of states')

plt.savefig('imgs/parsec.pdf')